# Bibliotecas

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from obterDados import obterSimbolo
import numpy as np
from tqdm import tqdm

# Obtendo dados

In [ ]:
rawDataM5 = obterSimbolo('WDO$')
rawDataM5

In [ ]:
rawDataM5 = rawDataM5.drop(columns=['spread'])
rawDataM5['hour'] = rawDataM5.index.hour
rawNP = rawDataM5.to_numpy()
rawNP

# Tratamento de Dados

### Gerando colunas com preços passados

In [ ]:
nPassado = 1000
nFuturo = 24
histNp = np.zeros((len(rawNP)-(nPassado+nFuturo), nPassado, 7))
for i in tqdm(range(len(histNp))):
    histNp[i] = rawNP[i:i+nPassado]

### Achar quando comprar, vender ou fazer nada

In [ ]:
pontos = 30
actions = np.zeros((len(histNp)))
for i in tqdm(range(len(actions))):
    ultimoPreco = histNp[i, -1, 3]
    for j in range(nFuturo):
        # nao fazer nada se for mais de 14hs
        if rawNP[nPassado+i+j, -1] > 14:
            break
        # comprar
        if rawNP[nPassado+i+j, 1] >= ultimoPreco + pontos:
            actions[i] = 1
            break
        # vender
        if rawNP[nPassado+i+j, 2] <= ultimoPreco - pontos:
            actions[i] = 2
            break

### Normalização

In [ ]:
histNpNorm = np.zeros(histNp.shape)
for i in tqdm(range(len(histNpNorm))):
    # preços
    vmax = histNp[i, :, :4].max()
    vmin = histNp[i, :, :4].min()
    histNpNorm[i, :, :4] = (histNp[i, :, :4] - vmin) / (vmax - vmin)
    # tick volume
    vmax = histNp[i, :, 4].max()
    vmin = histNp[i, :, 4].min()
    histNpNorm[i, :, 4] = (histNp[i, :, 4] - vmin) / (vmax - vmin)
    # real volume
    vmax = histNp[i, :, 5].max()
    vmin = histNp[i, :, 5].min()
    histNpNorm[i, :, 5] = (histNp[i, :, 5] - vmin) / (vmax - vmin)
# hora
histNpNorm[:, :, 6] = histNp[:, :, 6] / 24
histNpNorm[0]

### Separar dados de treino e teste

In [ ]:
splitDays = 108 * 200
x_train_class = histNpNorm[:-splitDays]
x_test_class = histNpNorm[-splitDays:]
y_train_class = actions[:-splitDays]
y_test_class = actions[-splitDays:]

# Criar Modelo

### Lazy Classifier

In [ ]:
from lazypredict.Supervised import LazyClassifier

lazyReg = LazyClassifier()
lazyReg.fit(x_train_class, x_test_class, y_train_class, y_test_class)

### Gerando pesos das classes

In [ ]:
import numpy as np

pesos = y_train_class.copy()
maximaOcorrencia = -1
for i in range(3):
    print(y_train_class[y_train_class == i].count())
    if y_train_class[y_train_class == i].count() > maximaOcorrencia:
        maximaOcorrencia = y_train_class[y_train_class == i].count()
for i in range(3):
    pesos[pesos == i] = maximaOcorrencia / (len(pesos) * y_train_class[y_train_class == i].count())
pesos

### Criando um dos modelos encontrados

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

modelo = MLPClassifier(verbose=True)
# modelo = AdaBoostClassifier()
# modelo.fit(x_train_class, y_train_class, sample_weight=pesos)
modelo.fit(x_train_class, y_train_class)

### Salvar Modelo de Classificação

In [ ]:
import pickle

pickle.dump(modelo, open('modeloClassificadorMLP.pickle', 'wb'))
# pickle.dump(modelo, open('modeloClassificadorAdaBoost.pickle', 'wb'))

# Pontuação de treino e teste

In [ ]:
from sklearn.metrics import classification_report
from pickle import load

modelo = load(open('modeloClassificadorAdaBoost.pickle', 'rb'))
print('Treino: ', classification_report(y_train_class, modelo.predict(x_train_class)))
print('Teste: ', classification_report(y_test_class, modelo.predict(x_test_class)))